# dengue-anvil_classifier_fromJSON
This classifier uses data from JSON delivered on October 2019 (dengue_filtered.json. This database has the most important features, and the JSON has filtered data from the complete JSON version (dengue_fixed.json): it lacks data of extended tweets (more than 140 characters) and has too few features to work with.

In [1]:
import pandas as pd
import numpy as np
import pickle
import datetime
from datetime import timedelta, date
import time
import string
import re

import json
#import ijson
#import bz2
import os
from pprint import pprint
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# configure MongoDB

In [2]:
import pymongo
from pymongo import MongoClient
import dns
client = MongoClient()

In [3]:
db = client.twitter
collection_ufmg = db.ufmg_filtered

In [105]:
path = '../../inputs/'
outputs = '../outputs/'

# retrieve data from Mongo

In [103]:
time.strptime('2017-01-17 14:36:26', '%Y-%m-%d %H:%M:%S')

time.struct_time(tm_year=2017, tm_mon=1, tm_mday=17, tm_hour=14, tm_min=36, tm_sec=26, tm_wday=1, tm_yday=17, tm_isdst=-1)

In [108]:
time.mktime(time.strptime('2012-12-13T23:29:56.000000'[:10], '%Y-%m-%d'))

1355367600.0

In [30]:
file_len = 7503436
objects = collection_ufmg.find({})
tweets_list = []
tweets_df = pd.DataFrame()
count = 0
for obj in objects:
    date = obj['date']
    if isinstance(date, int):
        date = date/1000
    else: date = time.mktime(time.strptime(date[:10], '%Y-%m-%d'))
    if date >= 1451617260: # 1451617260 = 2016-01-01
        obj['date'] = time.strftime('%Y-%m-%d', time.localtime(date))
        obj['text'] = re.sub(r'\\', '', obj['text'])
        if 'extended_tweet' in obj: 
            obj['extended_tweet'] = re.sub(r'\\', '', obj['extended_tweet']['full_text'])
        tweets_list.append(obj)

    count += 1
    if count % (int(file_len/1000)) == 0:
        if tweets_list: tweets_df = tweets_df.append(tweets_list, ignore_index=True)
        tweets_list = []
        clear_output()
        
        frac = count/file_len*100
        print("%.1f" % frac, "% done", sep="")

100.0% done


In [63]:
print(len(tweets_df))
tweets_df.head()

3651335


,_id,class_campanha,class_exp_pessoal,class_informacao,class_opiniao,class_parodia,coordinates,date,extended_tweet,lang,location,place,screen_name,text,user_id
0,682759181129150464,0.196747,0.295045,0.147560,0.163967,0.196680,NaN,2016-01-01,NaN,pt,"Cachoeira Paulista, Sao Paulo",NaN,PrisGaia,( musica dos vizinhos zika aq da comunidade),415033519
1,682759459882577920,0.045414,0.326786,0.245211,0.117666,0.264923,NaN,2016-01-01,NaN,pt,NaN,NaN,hemmingselvagem,"Sei que estou atrasada, mas FELIZ ANO NOVOOO. ...",3418723343
2,682759477951606785,0.093453,0.516996,0.230573,0.060079,0.098899,NaN,2016-01-01,NaN,pt,NaN,NaN,lariradavelli,O ano ja comeca assim: eu em casa com suspeita...,4107296837
3,682759587016097793,0.162586,0.321453,0.180436,0.131999,0.203527,NaN,2016-01-01,NaN,pt,"Pelotas, Rio Grande do Sul",NaN,fascinadona,@vitoriaawg imagina qnd a lua entrar em aquari...,375948921
4,682760489869393920,0.084446,0.270035,0.000000,0.341756,0.303763,NaN,2016-01-01,NaN,pt,Rocinha,NaN,carlosh89252899,O mais zika e quem tem palavra e nao que vende po,598670092


# filter by date

reference: https://portalarquivos2.saude.gov.br/images/pdf/2018/agosto/21/Publicacao-BE-2018-SE-30.pdf

* zika
    * peak year = 2016 
    * peak week = week 7 = 14-21/2/2016
    * occurrence timeframe = 7-21 = 14/2 a 28/5
* chikungunya
    * peak year = 2017 
    * peak week = week 17 = 23-30/4/2017
    * occurrence timeframe = 1-33 = 1/1 a 20/8
* dengue
    * peak year = 2016, but e used 2018 to add variability from other years
    * occurrence timeframe = 14-21 = 1/4 a 26/5

In [108]:
tweets_df = tweets_df_bak.copy()

removing 2018 for now because I want to compare with older anvil_input.

notice that on onder input, I chose different dates:
```python
data = data.loc[(data['date'] >= '2018-04-04 00:00:01') & (data['date'] <= '2018-05-27 23:59:59')]
```

In [109]:
tweets_df = tweets_df[(tweets_df['date'] >= '2016-02-14 00:00:01') & (tweets_df['date'] <= '2016-05-28 23:59:59') \
                 | (tweets_df['date'] >= '2017-01-01 00:00:01') & (tweets_df['date'] <= '2017-08-20 23:59:59') \
#                  | (tweets_df['date'] >= '2018-04-01 00:00:01') & (tweets_df['date'] <= '2018-05-26 23:59:59') \
    ]
tweets_df = tweets_df[tweets_df['lang'] == 'pt']

In [110]:
print(len(tweets_df))
tweets_df.tail()

1458159


,_id,class_campanha,class_exp_pessoal,class_informacao,class_opiniao,class_parodia,coordinates,date,extended_tweet,lang,location,place,screen_name,text,user_id
3126760,899464832973508608,0.058373,0.120385,0.185085,0.049518,0.586640,NaN,2017-08-20,NaN,pt,"Gravatai, Brasil",NaN,ueler_santos,Whats sempre parado daqui a pouco cria dengue,3436115489
3126761,899464882868936704,0.058373,0.120385,0.185085,0.049518,0.586640,NaN,2017-08-20,NaN,pt,"Gravatai, Rio Grande do Sul",NaN,thainaSCI,RT @ueler_santos: Whats sempre parado daqui a ...,363373624
3126762,899464967791022080,0.037072,0.469621,0.000000,0.246943,0.246364,NaN,2017-08-20,NaN,pt,"Sao Paulo, Brasil",NaN,LiviaaDecesary,"Eu amo meu primo ! Eu me racho com ele, Zika p...",3045146361
3126763,899466012080427008,0.169007,0.392241,0.329129,0.064071,0.045552,NaN,2017-08-20,NaN,pt,"Jardim Catarina, Sao Goncalo",NaN,dgmorete,"RT @cartolandobr: Cheguei do trabalho, mas que...",2892216646
3126764,899466053079748608,0.055360,0.082978,0.251239,0.099546,0.510877,NaN,2017-08-20,NaN,pt,Green Coast - Rio de Janeiro,NaN,RomeeroSiilva,RT @italoffc_: Racista e igual mosquito da den...,754310319338557441


In [83]:
data_json = tweets_df.to_json(orient='records')

In [84]:
file = os.path.join(path,'anvil_input_allfeatures.json')
json.dump(json.loads(data_json), open(file, 'w'))

In [ ]:
data = data.iloc[:, 0:3]

In [85]:
sample = tweets_df.sample(3000)

In [87]:
len(sample)

3000

In [102]:
set(list(tweets_df['lang']))

{nan, 'pt'}

In [ ]:
tweets_object = []
count = 1027
sample = tweets_df.sample(3000)
for index, row in sample.iterrows():
    sample_dict = {}
    sample_dict['id'] = row['_id']
    if isinstance(row['extended_tweet'], str):
        text = re.sub('\n+', ' ', row['extended_tweet'])
        #print(text)
    else:
        text = re.sub('\n+', ' ', row['text'])
    #for symbol in ['¿', '¡', 'ñ']:
    #    if symbol in text: continue
    #text = remove_special_char(text)
    #sample_dict['message'] = text.encode('ascii', errors='ignore').decode('ascii') # remove special characters
    sample_dict['message'] = text
    sample_dict['count'] = count
    tweets_object.append(sample_dict)
    count += 1

In [101]:
tweets_object

[{'id': 700647364294365184,
  'message': 'ZIKA VIRUS - PROFECIA &amp; CUMPRIMENTO - PROFETA DAVID OWUOR https://t.co/hLh2am5jsf',
  'count': 1027},
 {'id': 704029311238586368,
  'message': 'Jovem do Japao e diagnosticado com Zika virus apos viagem ao Brasil https://t.co/LWh0NWW1Ve via @YahooBr',
  'count': 1028},
 {'id': 700683472067829760,
  'message': 'RT @luanadpla: @BertoldiBruno volta p caxias, por favor, ta contaminando a cidade, zika virus',
  'count': 1029},
 {'id': 699330572859023360,
  'message': 'Araraquara, SP, registra o primeiro caso de virus zika em gravida 35 anos: Paciente infectada esta com 36 sema... https://t.co/XP9ShkZo7S',
  'count': 1030},
 {'id': 818911537796698116,
  'message': '@ultraflorescent pode ter sido um mosquito da dengue ai ele vai te picar e vc vai ter zika interna',
  'count': 1031},
 {'id': 711968789923926016,
  'message': 'Preveno  https://t.co/1H3nP8048t',
  'count': 1032},
 {'id': 699649514328748033,
  'message': 'Se houvesse um minimo razoavel 

In [112]:
file = os.path.join(outputs, 'tweets_anvil_input_fromJSON.json')
json.dump(tweets_object, open(file, 'w'))